In [ ]:
%%capture
!pip3 install torch --extra-index-url https://download.pytorch.org/whl/cu116
!pip3 install rdkit
!pip3 install git+https://github.com/molecularsets/moses.git

In [ ]:
import sys
import os
import torch
torch.cuda.set_device(0)
import pandas as pd
import moses
from moses.utils import disable_rdkit_log
disable_rdkit_log()
import importlib

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
sys.path.insert(1, "/content/gdrive/My Drive/gentrl_v2")
import gentrl_v2

In [ ]:
os.chdir('/content/gdrive/My Drive/gentrl_v2')

In [ ]:
LATENT_SIZE = 8
LATENT_DESC = LATENT_SIZE * [('c', 10)]
FEATURE_DESC = [('c', 10), ('c', 10), ('d', 3), ('c', 10)]
BETA = 0.001
GAMMA = 0.95
# change according to the desired combination of data and VAE loss function
# as described in pretrain.ipynb
EXPERIMENT_NAME = "009_random_cvae_rnn_50k"
BATCH_SIZE = 128
NUM_ITERS = 40000
START_ITER = 1
LOAD_EPOCH = 400

In [ ]:
enc = gentrl_v2.RNNEncoder(latent_size=LATENT_SIZE, bidirectional=True, hidden_size=128)
dec = gentrl_v2.RNNDecoder(latent_input_size=LATENT_SIZE)
model = gentrl_v2.GENTRL(enc, dec, LATENT_DESC, FEATURE_DESC,  beta=BETA, gamma=GAMMA, state_dim=LATENT_SIZE, action_dim=8)
model.cuda();

In [ ]:
model.load('models/'+EXPERIMENT_NAME+'/'+str(LOAD_EPOCH))
print('Start new: models/'+EXPERIMENT_NAME+'/'+str(LOAD_EPOCH))
model.cuda();

In [ ]:
experts = pd.read_csv('./data/canonized/esr1.csv')
expert_smiles = experts['SMILES'].tolist()

In [ ]:
len(expert_smiles)

In [ ]:
decoys = pd.read_csv('./data/canonized/esr1_decoys.csv')
decoy_smiles = decoys['SMILES'].tolist()

In [ ]:
len(decoy_smiles)

In [ ]:
# copied from MOSES
def get_mol(smiles_or_mol):
    '''
    Loads SMILES/molecule into RDKit's object
    '''
    if isinstance(smiles_or_mol, str):
        if len(smiles_or_mol) == 0:
            return None
        mol = Chem.MolFromSmiles(smiles_or_mol)
        if mol is None:
            return None
        try:
            Chem.SanitizeMol(mol)
        except:
            return None
        return mol
    return smiles_or_mol

In [ ]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol
from rdkit import Chem
from rdkit.Chem import QED, Descriptors, RDConfig, MolFromSmiles, SanitizeMol, MolToSmiles, AddHs
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

from moses.utils import disable_rdkit_log
disable_rdkit_log()

def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])

# added QED to favor molecules with high predicted bioactivity
def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = QED.qed(mol)*(Descriptors.MolLogP(mol) - sascorer.calculateScore(mol) - get_num_rings_6(mol))
    return reward

In [ ]:
model.train_as_rl(reward_fn=penalized_logP,
                  train_name=EXPERIMENT_NAME,
                  num_iterations=NUM_ITERS,
                  batch_size=BATCH_SIZE,
                  lr_lp=1e-4, lr_dec=1e-5,
                  explore_perc=0.3)

In [ ]:
model.train_as_il(expert_smiles=expert_smiles,
                  train_name=EXPERIMENT_NAME,
                  num_iterations=NUM_ITERS,
                  batch_size=BATCH_SIZE,
                  compare_batch_size=BATCH_SIZE,
                  lr_lp=1e-4, lr_dec=1e-5,
                  initial_explore_perc=0.9, final_explore_perc=0.3)

In [ ]:
model.train_as_il2(expert_smiles=expert_smiles,
                  train_name=EXPERIMENT_NAME,
                  num_iterations=NUM_ITERS,
                  batch_size=BATCH_SIZE,
                  compare_batch_size=BATCH_SIZE,
                  lr_lp=1e-4, lr_dec=1e-5,
                  initial_explore_perc=0.9, final_explore_perc=0.3)

In [ ]:
model.train_as_il3(expert_smiles=expert_smiles,
                  train_name=EXPERIMENT_NAME,
                  num_iterations=NUM_ITERS,
                  batch_size=BATCH_SIZE,
                  compare_batch_size=BATCH_SIZE,
                  lr_lp=1e-4, lr_dec=1e-5,
                  initial_explore_perc=0.9, final_explore_perc=0.3)

In [ ]:
model.train_as_il4(expert_smiles=expert_smiles,
                  train_name=EXPERIMENT_NAME,
                  num_iterations=NUM_ITERS,
                  batch_size=BATCH_SIZE,
                  compare_batch_size=BATCH_SIZE,
                  lr_lp=1e-4, lr_dec=1e-5,
                  initial_explore_perc=0.3, final_explore_perc=0.3)

In [ ]:
model.train_as_il5(expert_smiles=expert_smiles,
                  train_name=EXPERIMENT_NAME,
                  num_iterations=NUM_ITERS,
                  batch_size=BATCH_SIZE,
                  compare_batch_size=BATCH_SIZE,
                  lr_lp=1e-4, lr_dec=1e-5,
                  initial_explore_perc=0.9, final_explore_perc=0.3)